In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import numpy as np

In [ ]:
# Create variables for each of the data files
nyt_data = 'https://group3data.s3.us-west-2.amazonaws.com/us-counties.csv'
hospital_data = 'https://group3data.s3.us-west-2.amazonaws.com/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv'
vaccination_data = 'https://group3data.s3.us-west-2.amazonaws.com/COVID-19_Vaccinations_in_the_United_States_County.csv'
merged_data = 'https://group3data.s3.us-west-2.amazonaws.com/merged_data.csv'

In [ ]:
# read in data from S3 bucket and create dataframes
nyt_data_df = pd.read_csv(nyt_data)
hospital_data_df = pd.read_csv(hospital_data)
vaccination_data_df = pd.read_csv(vaccination_data)
merged_data_df = pd.read_csv(merged_data)

Create the machine learning model

In [ ]:
# Import additional dependecies for the model
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# Use a test data set that is binary to create the model
# We will import our data into the model later
from sklearn.datasets import make_blobs
X, y = make_blobs(centers=2, random_state=42)

In [ ]:
# Plot the blobs to visualize the 2 data centers
plt.scatter(X[:, 0], X[:, 1], c=y)

In [ ]:
# Split, test, and train the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, stratify=y)

In [ ]:
# Create the logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

In [ ]:
# use fit() to train the logistic regression model
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions of the data
predictions = classifier.predict(X_test)

# Visualize the predictions in a easy to read dataframe
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).head()

In [ ]:
# Get the accuracy score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

In [ ]:
# Now we want to through new data into the model to see what it would predict
new_data = np.array([[-2, 6]])
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.scatter(new_data[0, 0], new_data[0, 1], c="r", marker="o", s=100)
plt.show()

In [ ]:
# pass the 'new_data' into the predict method of the classifier model that was instantiated earlier
predictions = classifier.predict(new_data)
print("Classes are either 0 (purple) or 1 (yellow)")
print(f"The new point was classified as: {predictions}")

Pass in the merged_data_df to the ML Model

In [ ]:
merged_data_df['hospital_bed_usage'] = merged_data_df['all_adult_hospital_inpatient_bed_occupied_7_day_sum']/merged_data_df['inpatient_beds_7_day_sum']
merged_data_df

In [ ]:
merged_data_df['hospital_bed_usage'].loc[merged_data_df['hospital_bed_usage'] < 0.85] = 0
merged_data_df['hospital_bed_usage'].loc[merged_data_df['hospital_bed_usage'] >= 0.85] = 1
merged_data_df

In [ ]:
# Get the counts for overloaded or not
merged_data_df.dropna(inplace=True)
ml_model_df = merged_data_df
ml_model_df.drop(['fips_date', 'fips', 'date'], axis=1, inplace=True)
merged_data_df['hospital_bed_usage'].value_counts()

In [ ]:
# Create the model variables
X = ml_model_df.drop(['hospital_bed_usage'], axis=1)
y = ml_model_df['hospital_bed_usage']

In [ ]:
# Split, Test, and Train the dataset
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, stratify=y)

In [ ]:
# Create the logistic regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

In [ ]:
# use fit() to train the logistic regression model
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions of the data
predictions = classifier.predict(X_test)

In [ ]:
# Get the accuracy score of the model
accuracy_score(y_test, predictions)

In [ ]:
# Print the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, predictions)
print(matrix)

In [ ]:
# Get the classification report
report = classification_report(y_test, predictions)
print(report)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
Counter(y_resampled)

In [ ]:
# Rerun the model
model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(y_test, y_pred))

In [ ]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampled)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
bed_capacity = balanced_accuracy_score(y_test, y_pred)
bed_capacity

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Create df for confusion matrix
cm_df = pd.DataFrame(cm, index=["Actual full", "Actual spacious"], columns=["Predicted full", "Predicted spacious"])

cm_df

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {bed_capacity}")
print("Classification Report")
print(classification_report(y_test, y_pred))

SMOTEENN

In [ ]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
bed_capacity = balanced_accuracy_score(y_test, y_pred)
bed_capacity

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Create df for confusion matrix
cm_df = pd.DataFrame(cm, index=["Actual full", "Actual spacious"], columns=["Predicted full", "Predicted spacious"])
cm_df